In [1]:
import sys
from dotenv import load_dotenv
import pandas as pd
sys.path.insert(1, '/home/jabez/week_11/Contract-Advisor-RAG')
load_dotenv()
sys.path.insert(1, '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/scripts')
import file_loader 
import pipelines 
import evaluation

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/data/cnn_dailymail_3.0.0.csv'
data = file_loader.load_csv(file_path)

In [3]:
vectorstore_character = file_loader.character_text_splitter(data)

In [4]:
syntetic_test_data =pd.read_csv('/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data.csv')

In [5]:
# Setting retriever
retriver = vectorstore_character.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [7]:
file_path = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/multiquery_answer.csv'
multiquery_answer = evaluation.adding_answer_to_testdata(syntetic_test_data,pipelines.multi_query_pipeline, vectorstore_character, retriver,file_path)

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 86.44ba/s]


In [8]:
multiquery_rag_evaluation_result = evaluation.ragas_evaluator(multiquery_answer)

Evaluating: 100%|██████████| 40/40 [00:19<00:00,  2.06it/s]


In [10]:
# Evaluation mean
def evaluation_mean(evaluation_result):
    context_precision = round(evaluation_result['context_precision'].mean()*100,2)
    faithfulness = round(evaluation_result['faithfulness'].mean()*100,2)
    answer_relevancy = round(evaluation_result['answer_relevancy'].mean()*100,2)
    context_recall = round(evaluation_result['context_recall'].mean()*100,2)
    print(f'context_precision: {context_precision}%, faithfulness: {faithfulness}%, answer_relevancy: {answer_relevancy}%, context_recall: {context_recall}%')

result = evaluation_mean(multiquery_rag_evaluation_result)

context_precision: 86.5%, faithfulness: 84.75%, answer_relevancy: 94.38%, context_recall: 96.67%
